In [31]:
import pandas as pd
import nltk
from string import punctuation
import re
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.model_selection import KFold,cross_val_score,train_test_split
from nltk.stem import WordNetLemmatizer
import spacy
from sklearn.naive_bayes import MultinomialNB
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from tensorflow.keras import regularizers

In [ ]:
nltk.download("all")

In [3]:
data =  pd.read_csv(r"C:\Users\USUARIO\Downloads\Musical_instruments_reviews.csv")
data_cleaned = data.dropna(subset=["reviewText"])
data_cleaned.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A2IBPI20UZIR0U,1384719342,"cassandra tu ""Yeah, well, that's just like, u...","[0, 0]","Not much to write about here, but it does exac...",5.0,good,1393545600,"02 28, 2014"
1,A14VAT5EAX3D9S,1384719342,Jake,"[13, 14]",The product does exactly as it should and is q...,5.0,Jake,1363392000,"03 16, 2013"
2,A195EZSQDW3E21,1384719342,"Rick Bennette ""Rick Bennette""","[1, 1]",The primary job of this device is to block the...,5.0,It Does The Job Well,1377648000,"08 28, 2013"
3,A2C00NNG1ZQQG2,1384719342,"RustyBill ""Sunday Rocker""","[0, 0]",Nice windscreen protects my MXL mic and preven...,5.0,GOOD WINDSCREEN FOR THE MONEY,1392336000,"02 14, 2014"
4,A94QU4C90B1AX,1384719342,SEAN MASLANKA,"[0, 0]",This pop filter is great. It looks and perform...,5.0,No more pops when I record my vocals.,1392940800,"02 21, 2014"


In [4]:
print(data_cleaned.info())

<class 'pandas.core.frame.DataFrame'>
Index: 10254 entries, 0 to 10260
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   reviewerID      10254 non-null  object 
 1   asin            10254 non-null  object 
 2   reviewerName    10227 non-null  object 
 3   helpful         10254 non-null  object 
 4   reviewText      10254 non-null  object 
 5   overall         10254 non-null  float64
 6   summary         10254 non-null  object 
 7   unixReviewTime  10254 non-null  int64  
 8   reviewTime      10254 non-null  object 
dtypes: float64(1), int64(1), object(7)
memory usage: 801.1+ KB
None


In [13]:
#Removing stop-words, punctuations and lemmatizing the words to increase model performance
stop_words = set(nltk.corpus.stopwords.words("english"))
stop_words.update(punctuation)
tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
lemmatizer = WordNetLemmatizer()
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN  
def preprocess_text(text):
    if isinstance(text,str):
        tokens = tokenizer.tokenize(re.sub(r'[^\w\s]|\d|_*_' ,'',text.lower()))
        pos_tags = nltk.pos_tag(tokens)
        temp = [lemmatizer.lemmatize(i,get_wordnet_pos(tag)) for i,tag in pos_tags if i not in stop_words]
        return " ".join(temp)
    return ""
data_cleaned.loc[:,"review_v2"] = data_cleaned["reviewText"].apply(preprocess_text)
print(data_cleaned["review_v2"][0])

much write exactly suppose filter pop sound recording much crisp one low price pop filter amazon might well buy honestly work despite pricing


In [79]:
#Vectorizing texts with bag-of-words
bow_vectorizer = CountVectorizer()
bow = bow_vectorizer.fit_transform(data_cleaned["review_v2"])
bow_dataframe = pd.DataFrame.sparse.from_spmatrix(bow,columns=bow_vectorizer.get_feature_names_out())
bow_dataframe

,aa,aaa,aaas,ab,aback,abalone,abalonefender,abandon,abbe,abcd,...,zoneso,zoom,zoomnow,zt,ztif,ztupdate,zune,zvex,zylgian,zz
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10249,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10250,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10251,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10252,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
#Vectorizing texts with TFIDF 
tfidf_vectorizer = TfidfVectorizer()
vector = tfidf_vectorizer.fit_transform(data_cleaned["review_v2"])
tfidf_dataframe = pd.DataFrame(vector.todense(),columns=tfidf_vectorizer.get_feature_names_out())
tfidf_dataframe

,aa,aaa,aaas,ab,aback,abalone,abalonefender,abandon,abbe,abcd,...,zoneso,zoom,zoomnow,zt,ztif,ztupdate,zune,zvex,zylgian,zz
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10249,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10250,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10251,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10252,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
#Defining model and cross-validation strategy
kf = KFold(n_splits=5,shuffle=True,random_state=42)
model = MultinomialNB()

In [18]:
#Testing bag-of-words multinomial model performance
results_bow = cross_val_score(model,bow_dataframe,data_cleaned["overall"],cv=kf,n_jobs=-1)
print(f"The mean accuracy of the bag-of-words multinomial model was {(results_bow.mean()*100):.3f}% ")

The mean accuracy of the bag-of-words multinomial model was 67.183% 


In [19]:
#Testing TFIDF multinomial model performance
results_tfidf = cross_val_score(model,tfidf_dataframe,data_cleaned["overall"],cv=kf,n_jobs=-1)
print(f"The mean accuracy of the TFIDF multinomial model was {(results_tfidf.mean()*100):.3f}% ")

The mean accuracy of the TFIDF multinomial model was 67.593% 


In [20]:
#Vectorizing texts with tensorflow
tf_tokenizer = Tokenizer()
tf_tokenizer.fit_on_texts(data_cleaned["review_v2"])
sequence = tf_tokenizer.texts_to_sequences(data_cleaned["review_v2"])
sequence[0]

[26,
 661,
 286,
 410,
 474,
 361,
 3,
 333,
 26,
 997,
 4,
 95,
 19,
 361,
 474,
 179,
 160,
 10,
 14,
 881,
 12,
 1121,
 2751]

In [78]:
#Creating a SimpleRNN with embedding layer
max_length = 10
padded_sequence = tf.keras.preprocessing.sequence.pad_sequences(sequence,maxlen=max_length,padding="post")
vocab_size=23889
embedding_dim=100
model_simple = tf.keras.models.Sequential()
model_simple.add(tf.keras.layers.Embedding(input_dim=vocab_size,output_dim=embedding_dim,input_length=max_length))
model_simple.add(tf.keras.layers.SimpleRNN(64,activation="relu",return_sequences=True))
model_simple.add(tf.keras.layers.Dropout(0.5))
model_simple.add(tf.keras.layers.SimpleRNN(32,activation="relu"))
model_simple.add(tf.keras.layers.Dropout(0.5))
model_simple.add(tf.keras.layers.Dense(6,activation="softmax"))
model_simple.compile(loss="sparse_categorical_crossentropy",metrics=["accuracy"],optimizer="adam")

x_full,x_test,y_full,y_test=train_test_split(padded_sequence,data_cleaned["overall"],random_state=42,test_size=0.2)
x_train,x_valid,y_train,y_valid = train_test_split(x_full,y_full,random_state=42,test_size=0.2)
model_simple.fit(x_train,y_train,epochs=10,validation_data=(x_valid,y_valid))
test_loss,test_acc = model_simple.evaluate(x_test,y_test)
print(f"The test accuracy was {100*test_acc:.3f}%")

Epoch 1/10
206/206 [==============================] - 18s 76ms/step - loss: 1.1507 - accuracy: 0.6237 - val_loss: 0.9603 - val_accuracy: 0.6520
Epoch 2/10
206/206 [==============================] - 15s 74ms/step - loss: 0.9214 - accuracy: 0.6693 - val_loss: 1.0178 - val_accuracy: 0.6520
Epoch 3/10
206/206 [==============================] - 15s 71ms/step - loss: 0.7902 - accuracy: 0.6792 - val_loss: 1.0318 - val_accuracy: 0.6545
Epoch 4/10
206/206 [==============================] - 15s 73ms/step - loss: 0.6835 - accuracy: 0.7118 - val_loss: 1.1628 - val_accuracy: 0.6222
Epoch 5/10
206/206 [==============================] - 15s 74ms/step - loss: 0.5898 - accuracy: 0.7562 - val_loss: 1.5045 - val_accuracy: 0.6015
Epoch 6/10
206/206 [==============================] - 15s 73ms/step - loss: 0.4833 - accuracy: 0.8092 - val_loss: 2.2063 - val_accuracy: 0.5990
Epoch 7/10
206/206 [==============================] - 15s 71ms/step - loss: 0.3908 - accuracy: 0.8552 - val_loss: 1.7009 - val_accuracy:

In [71]:
#Creating RNN Unidirectional LSTM with embedding layer
max_length = 10
padded_sequence = tf.keras.preprocessing.sequence.pad_sequences(sequence,maxlen=max_length,padding="post")
vocab_size=23889
embedding_dim=100
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Embedding(input_dim=vocab_size,output_dim=embedding_dim,input_length=max_length))
model.add(tf.keras.layers.LSTM(64,return_sequences=True,kernel_regularizer=regularizers.l2(0.06)))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.LSTM(64,kernel_regularizer=regularizers.l2(0.06)))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(6,activation="softmax"))
model.compile(loss="sparse_categorical_crossentropy",metrics=["accuracy"],optimizer="adam")

x_full,x_test,y_full,y_test=train_test_split(padded_sequence,data_cleaned["overall"],random_state=42,test_size=0.2)
x_train,x_valid,y_train,y_valid = train_test_split(x_full,y_full,random_state=42,test_size=0.2)
model.fit(x_train,y_train,epochs=10,validation_data=(x_valid,y_valid))
test_loss,test_acc = model.evaluate(x_test,y_test)
print(f"The test accuracy was {100*test_acc:.3f}%")

Epoch 1/10
206/206 [==============================] - 7s 19ms/step - loss: 3.8120 - accuracy: 0.6713 - val_loss: 1.0079 - val_accuracy: 0.6520
Epoch 2/10
206/206 [==============================] - 3s 15ms/step - loss: 0.9649 - accuracy: 0.6743 - val_loss: 0.9881 - val_accuracy: 0.6520
Epoch 3/10
206/206 [==============================] - 3s 16ms/step - loss: 0.9649 - accuracy: 0.6743 - val_loss: 1.0101 - val_accuracy: 0.6520
Epoch 4/10
206/206 [==============================] - 3s 16ms/step - loss: 0.9633 - accuracy: 0.6743 - val_loss: 0.9879 - val_accuracy: 0.6520
Epoch 5/10
206/206 [==============================] - 3s 16ms/step - loss: 0.9625 - accuracy: 0.6743 - val_loss: 0.9870 - val_accuracy: 0.6520
Epoch 6/10
206/206 [==============================] - 3s 16ms/step - loss: 0.9604 - accuracy: 0.6743 - val_loss: 0.9892 - val_accuracy: 0.6520
Epoch 7/10
206/206 [==============================] - 3s 16ms/step - loss: 0.9604 - accuracy: 0.6743 - val_loss: 0.9901 - val_accuracy: 0.6520

In [61]:
#Creating RNN Bidirectional LSTM with embedding layer
max_length = 10
padded_sequence = tf.keras.preprocessing.sequence.pad_sequences(sequence,maxlen=max_length,padding="post")
vocab_size=23889
embedding_dim=100
model_bi = tf.keras.models.Sequential()
model_bi.add(tf.keras.layers.Embedding(input_dim=vocab_size,output_dim=embedding_dim,input_length=max_length))
model_bi.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,return_sequences=True,kernel_regularizer=regularizers.l2(0.035))))
model_bi.add(tf.keras.layers.Dropout(0.2))
model_bi.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,kernel_regularizer=regularizers.l2(0.035))))
model_bi.add(tf.keras.layers.Dropout(0.3))
model_bi.add(tf.keras.layers.Dense(6,activation="softmax"))
model_bi.compile(loss="sparse_categorical_crossentropy",metrics=["accuracy"],optimizer="adam")

x_full,x_test,y_full,y_test=train_test_split(padded_sequence,data_cleaned["overall"],random_state=42,test_size=0.2)
x_train,x_valid,y_train,y_valid = train_test_split(x_full,y_full,random_state=42,test_size=0.2)
model_bi.fit(x_train,y_train,epochs=10,validation_data=(x_valid,y_valid))
test_loss,test_acc = model_bi.evaluate(x_test,y_test)
print(f"The test accuracy was {100*test_acc:.3f}%")

Epoch 1/10
206/206 [==============================] - 13s 32ms/step - loss: 5.0058 - accuracy: 0.6714 - val_loss: 1.0092 - val_accuracy: 0.6520
Epoch 2/10
206/206 [==============================] - 5s 23ms/step - loss: 0.9729 - accuracy: 0.6743 - val_loss: 1.0158 - val_accuracy: 0.6520
Epoch 3/10
206/206 [==============================] - 5s 22ms/step - loss: 0.9665 - accuracy: 0.6743 - val_loss: 0.9963 - val_accuracy: 0.6520
Epoch 4/10
206/206 [==============================] - 5s 22ms/step - loss: 0.9657 - accuracy: 0.6743 - val_loss: 0.9896 - val_accuracy: 0.6520
Epoch 5/10
206/206 [==============================] - 5s 23ms/step - loss: 0.9649 - accuracy: 0.6743 - val_loss: 0.9914 - val_accuracy: 0.6520
Epoch 6/10
206/206 [==============================] - 5s 23ms/step - loss: 0.9624 - accuracy: 0.6743 - val_loss: 0.9882 - val_accuracy: 0.6520
Epoch 7/10
206/206 [==============================] - 5s 23ms/step - loss: 0.9635 - accuracy: 0.6743 - val_loss: 0.9926 - val_accuracy: 0.652